# 一致性测试

对于大多数问题，多个磁通状态可以达到相同的最优，因此我们试图获得一致的网络。我们的意思是，网络中将存在多个阻塞反应，从而导致这种不一致。为解决这个问题，我们使用的算法可以检测所有被阻断的反应，并为我们提供一致的网络。

Let us take a toy network, like so:

\begin{align}
v_1 &: {} \rightarrow 2A \\
v_2 &: A \leftrightarrow B \\
v_3 &: A \rightarrow D \\
v_4 &: A \rightarrow C \\
v_5 &: C \rightarrow D \\
v_6 &: D \rightarrow
\end{align}

Here, $v_{x}$, where $x \in \{1, 2, \ldots, 6\}$ represent the flux carried by the reactions as shown above.

In [1]:
import cobra

In [2]:
test_model = cobra.Model("test_model")
v1 = cobra.Reaction("v1")
v2 = cobra.Reaction("v2")
v3 = cobra.Reaction("v3")
v4 = cobra.Reaction("v4")
v5 = cobra.Reaction("v5")
v6 = cobra.Reaction("v6")

test_model.add_reactions([v1, v2, v3, v4, v5, v6])

v1.reaction = "-> 2 A"
v2.reaction = "A <-> B"
v3.reaction = "A -> D"
v4.reaction = "A -> C"
v5.reaction = "C -> D"
v6.reaction = "D ->"

v1.bounds = (0.0, 3.0)
v2.bounds = (-3.0, 3.0)
v3.bounds = (0.0, 3.0)
v4.bounds = (0.0, 3.0)
v5.bounds = (0.0, 3.0)
v6.bounds = (0.0, 3.0)

test_model.objective = v6

unknown metabolite 'A' created
unknown metabolite 'B' created
unknown metabolite 'D' created
unknown metabolite 'C' created


## 使用 FVA

我们可以遵循的第一种方法是使用 FVA（通量可变性分析），在许多其他应用中，它用于检测阻塞反应。 `cobra.flux_analysis.find_blocked_reactions()`  函数将返回使用 FVA 获得的所有被阻止反应的列表。

In [3]:
cobra.flux_analysis.find_blocked_reactions(test_model)

['v2']

As we see above, we are able to obtain the blocked reaction, which in this case is $v_2$.
正如上述所示，我们能够通过在本例中的 $v_2$，来获得阻断反应。

## 使用FASTCC

在`cobrapy`中获得一致网络的第二种方法是使用 FASTCC。使用这种方法，您可以期望有效地获得准确一致的网络。有关该算法的更多详细信息，请参阅[Vlassis N, Pacheco MP, Sauter T (2014)](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1003424).

In [4]:
consistent_model = cobra.flux_analysis.fastcc(test_model)
consistent_model.reactions

[<Reaction v1 at 0x21cc26101d0>,
 <Reaction v3 at 0x21cc27caa50>,
 <Reaction v4 at 0x21cc27cabd0>,
 <Reaction v5 at 0x21cc27cad50>,
 <Reaction v6 at 0x21cc27caf10>]

与 FVA 方法类似，我们能够确定 $v_2$ 确实是阻断反应。